In [1]:
from networkx.algorithms.core import core_number
from sympy.abc import alpha

base_dir = '../../../../T5 EVO'

In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import cross_validate,train_test_split
from sklearn.linear_model import RidgeCV, Ridge
from sklearn.metrics import mean_squared_error
import scipy
import time
from sklearn.multioutput import MultiOutputRegressor
from scipy.io import loadmat
import ast
import matplotlib.pyplot as plt
import seaborn as sns
# import torch
from sklearn.decomposition import PCA
from itertools import combinations
from scipy.spatial.distance import pdist, squareform

seed = 2024

In [3]:
# def custom_ridge_cv_regression(X, y):
#     linreg = RidgeCV(alphas=(0.001,0.01,10,100.0,215.44,464.16,1000.0,2154.43,4641.59,10000.0,21544.35,46415.89,100000.0,215443.47,464158.88,1000000.0,2154434.69,4641588.83,10000000.0)
# )
#     linreg = MultiOutputRegressor(linreg,n_jobs=-1)
#     estimator = linreg.fit(X, y)
#     return estimator

In [4]:
def custom_ridge_regression(X, y,alpha):
    linreg = Ridge(alpha=alpha)
    linreg = MultiOutputRegressor(linreg,n_jobs=-1)

    estimator = linreg.fit(X, y)
    return estimator

In [5]:
def prepare_dataset(ds):
    if 'y' in ds.columns:
        ds['y'] = ds['y'].apply(ast.literal_eval)
    ds['embeddings'] = ds['embeddings'].apply(ast.literal_eval)
    return ds


In [6]:
def read_fmri(subject_id,selected_roi):
    parent_input_sagar_original = f'{base_dir}/fmri/Fahime/odor_responses_S1-3_regionized/odor_responses_S{subject_id}.mat'
    data1 = loadmat(parent_input_sagar_original)
    
    
    rois = {}
    rois['PirF'] = data1['odor_vals'][0][0]
    rois['PirT'] = data1['odor_vals'][0][1]
    rois['AMY'] = data1['odor_vals'][0][2]
    rois['OFC'] = data1['odor_vals'][0][3]
    
    for key in rois.keys():
        roi = rois[key]
        roi = np.moveaxis(roi, -1, 0)
        # roi = np.mean(roi,-1)
        rois[key] = roi
        
    data = rois[selected_roi]
    fmri = data.max(axis=2)
    return fmri

In [7]:
def read_fmri_singletrial(subject_id,selected_roi):
    parent_input_sagar_original = f'{base_dir}/fmri/average_of_singletrial/fmri_{subject_id}_{selected_roi}.csv'
    df = pd.read_csv(parent_input_sagar_original)

    fmri_array = df.pivot(index='CID', columns='voxel', values='fmri')
    fmri_array = fmri_array.to_numpy()
    # convert nan to 0
    fmri_array = np.nan_to_num(fmri_array)
    return fmri_array

In [8]:
def read_pom(CIDs,subject):

    
    embedding_open_pom = '/alignment_olfaction_datasets/curated_datasets/embeddings/pom/sagar_pom_embeddings_Apr17.csv'
    ds_pom = pd.read_csv(base_dir + embedding_open_pom)
    ds_pom = prepare_dataset(ds_pom)

    ds_pom = ds_pom.sort_values(by='CID')
    ds_pom = ds_pom[['CID', 'embeddings','y','subject']]
    ds_pom = ds_pom[ds_pom['subject'] == subject]
    ds_pom = ds_pom[ds_pom['CID'].isin(CIDs)]
    
    
    pom_embeddings = ds_pom['embeddings']
    pom_embeddings = np.array([np.array(x) for x in pom_embeddings])
    
    return pom_embeddings

In [9]:
def metrics_per_descriptor(X, y, linreg):
 
    """
    Calculate predictions, mean squared errors, and Pearson correlations for each descriptor.

    Parameters:
    X (array-like): Input features.
    y (array-like): True target values.
    linreg (object): Linear regression model with a predict method.

    Returns:
    tuple: A tuple containing predicted values, a list of MSEs, and a list of correlations.
    """
    predicted = linreg.predict(X)
    
    # Ensure y is 2D for consistent iteration
    y = y if len(y.shape) > 1 else y.reshape(-1, 1)
    predicted = predicted if len(predicted.shape) > 1 else predicted.reshape(-1, 1)
    
    

    mse_errors = [mean_squared_error(predicted[:, i], y[:, i]) for i in range(y.shape[1])]
    correlations = [scipy.stats.pearsonr(predicted[:, i], y[:, i]) for i in range(y.shape[1])]
    spearman_correlations = [scipy.stats.spearmanr(predicted[:, i], y[:, i]) for i in range(y.shape[1])]
    return predicted, mse_errors, correlations,spearman_correlations

In [10]:


def f_test(X, y, estimator):
    # Predicted values
    y_pred = estimator.predict(X)

    # Residual Sum of Squares (RSS)


    # Total Sum of Squares (SST)
    y_mean = np.mean(y, axis=0)
    # SST = np.sum((y - y_mean) ** 2)
    SSE = np.sum((y - y_pred) ** 2)
    SSM = np.sum((y_pred - y_mean) ** 2)
    SST = np.sum((y - y_mean) ** 2)

    # Regression Sum of Squares (SSR)
    # SSR = SST - SSE

    dfm = X.shape[1] - 1  # Degrees of freedom for the model
    dfe = X.shape[0] - X.shape[1]  # Degrees of freedom for the error
    dft = X.shape[0] - 1  # Total degrees of freedom

    msm = SSM / dfm  # Mean Squares for the model
    mse = SSE / dfe  # Mean Squares for the error
    mst = SST / dft  # Mean Squares for the total


    # Degrees of freedom
    # n = len(y)  # Number of samples
    # k = X.shape[1]  # Number of predictors (features)

    # F-statistic
    F =msm/mse

    # P-value
    from scipy.stats import f
    p_value = 1 - f.cdf(F, dfm, dfe)

    return F, p_value

In [11]:
# def train_and_eval_ntimes(X,y, times,n_components=None):
#     """
#     Train and evaluate a regression model using cross-validation.
#
#     Parameters:
#     data (DataFrame): Input data containing embeddings and fmri_average.
#     times (int): Number of cross-validation iterations.
#     n_components (int, optional): Number of components for dimensionality reduction.
#
#     Returns:
#     tuple: Contains CIDs, predicted values, test targets, runs, MSE errors, and correlations.
#     """
#     mse_errors_crossvalidated = []
#     correlations_crossvalidated = []
#     spearman_crossvalidated = []
#     predicteds = []
#     y_tests = []
#     runs_predictions = []
#     runs_metrics = []
#     voxels = []
#     voxels_predicteds = []
#     for i in range(times):
#         print(f"Processing run {i}")
#         X_train, X_test, y_train, y_test = train_test_split(
#             X, y, test_size=0.2, random_state=seed + i
#         )
#         if n_components:
#             pca = PCA(n_components=n_components)
#             #compute explained variance ratio
#             # X_train = pca.fit_transform(X_train)
#             # X_test = pca.transform(X_test)
#
#
#
#
#             X_train = pca.fit_transform(X_train)
#             X_test = pca.transform(X_test)
#
#
#         linreg = custom_ridge_cv_regression(X_train, y_train)
#         F, p_value = f_test(X_train, y_train, linreg)
#
#         if p_value > 0.05 or np.isnan(p_value):
#
#             continue
#         print(f"p_value {p_value}")
#
#         predicted, mse_errors, correlations,spearman_correlation = metrics_per_descriptor(X_test, y_test, linreg)
#
#
#         #flatten the predicted and y_test arrays
#         voxels_predicted = np.arange(predicted.shape[1])
#         predicted = np.concatenate(predicted)
#         y_test_flatten = np.concatenate(y_test)
#         mse_errors_crossvalidated.extend(mse_errors)
#         correlations_crossvalidated.extend(correlations)
#         spearman_crossvalidated.extend(spearman_correlation)
#         predicteds.extend(predicted)
#         y_tests.extend(y_test_flatten)
#
#         runs_metrics.extend([i] * y_test.shape[1])
#         runs_predictions.extend([i] * y_test.shape[0]*y_test.shape[1])
#
#         voxels.extend(np.arange(y.shape[1]))
#
#         voxels_predicted = np.repeat(voxels_predicted,y_test.shape[0])
#         voxels_predicteds.extend(voxels_predicted)
#
#     return predicteds, y_tests, runs_metrics,runs_predictions, mse_errors_crossvalidated, correlations_crossvalidated,spearman_crossvalidated,voxels,voxels_predicteds

In [12]:
def train_and_eval_ntimes_alphas(X,y, times,n_components=None):
    """
    Train and evaluate a regression model using cross-validation.

    Parameters:
    data (DataFrame): Input data containing embeddings and fmri_average.
    times (int): Number of cross-validation iterations.
    n_components (int, optional): Number of components for dimensionality reduction.

    Returns:
    tuple: Contains CIDs, predicted values, test targets, runs, MSE errors, and correlations.
    """
    mse_errors_crossvalidated = []
    correlations_crossvalidated = []
    spearman_crossvalidated = []
    predicteds = []
    y_tests = []
    runs_predictions = []
    runs_metrics = []
    voxels = []
    voxels_predicteds = []
    alphas = np.logspace(2,7,16)
    for i in range(times):
        correlations_per_alpha = []
        print(f"Processing run {i}")
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=0.2, random_state=seed + i
        )
        X_test, X_val, y_test, y_val = train_test_split(
            X_test, y_test, test_size=0.5, random_state=seed + i
        )
        if n_components:
            pca = PCA(n_components=n_components)
            X_train = pca.fit_transform(X_train)
            X_test = pca.transform(X_test)

        for alpha in alphas:
            linreg = custom_ridge_regression(X_train, y_train,alpha)
            # F, p_value = f_test(X_train, y_train, linreg)
            #
            # if p_value > 0.05 or np.isnan(p_value):
            #
            #     continue
            # print(f"p_value {p_value}")
            predicted, mse_errors, correlations,spearman_correlation = metrics_per_descriptor(X_test, y_test, linreg)
            correlations = np.asarray(correlations)
            correlations_per_alpha.append(correlations[:,0])
        if len(correlations_per_alpha) == 0:
            #create empty arrays
            predicted = np.zeros((y_test.shape[0],y_test.shape[1]))
            mse_errors = np.zeros(y_test.shape[1])
            correlations = np.zeros((y_test.shape[1],2))
            spearman_correlation = np.zeros((y_test.shape[1],2))


        elif len(correlations_per_alpha) == 1:
            best_alpha = alphas[0]
            linreg = custom_ridge_regression(X_train, y_train,best_alpha)
            predicted, mse_errors, correlations,spearman_correlation = metrics_per_descriptor(X_test, y_test, linreg)
        else:
            idx_best_alpha = np.argmax(np.mean(correlations_per_alpha, axis=1))
            best_alpha = alphas[idx_best_alpha]
            linreg = custom_ridge_regression(X_train, y_train,best_alpha)
            predicted, mse_errors, correlations,spearman_correlation = metrics_per_descriptor(X_test, y_test, linreg)





        #flatten the predicted and y_test arrays
        voxels_predicted = np.arange(predicted.shape[1])
        predicted = np.concatenate(predicted)
        y_test_flatten = np.concatenate(y_test)
        mse_errors_crossvalidated.extend(mse_errors)
        correlations_crossvalidated.extend(correlations)
        spearman_crossvalidated.extend(spearman_correlation)
        predicteds.extend(predicted)
        y_tests.extend(y_test_flatten)

        runs_metrics.extend([i] * y_test.shape[1])
        runs_predictions.extend([i] * y_test.shape[0]*y_test.shape[1])

        voxels.extend(np.arange(y.shape[1]))

        voxels_predicted = np.repeat(voxels_predicted,y_test.shape[0])
        voxels_predicteds.extend(voxels_predicted)

    return predicteds, y_tests, runs_metrics,runs_predictions, mse_errors_crossvalidated, correlations_crossvalidated,spearman_crossvalidated,voxels,voxels_predicteds

In [13]:
def pipeline(image,fmri, times=30,n_components=None):
    """
    Run the pipeline for a specific voxel and model, processing fMRI and embeddings data.

    Parameters:
    fmri_ev (DataFrame): fMRI event data.
    voxel (int): The voxel identifier.
    model_name (str): Name of the model being evaluated.
    input_file (str): Path to the input CSV file.
    times (int, optional): Number of cross-validation iterations. Default is 30.
    n_components (int, optional): Number of components for dimensionality reduction. Default is None.

    Returns:
    tuple: DataFrames containing predictions and metrics.
    """
    # Filter voxel-specific data
    y = fmri
    X = image
    
    

    # Train and evaluate the model
    results = train_and_eval_ntimes_alphas(X,y, times=times,n_components=n_components)
    predicteds, y_tests, runs_metrics,runs_predictions, mse_errors, correlations,spearman_correlations,voxel,voxels_predicteds = results
    if len(predicteds) == 0:
        return None,None
    predicteds = np.asarray(predicteds)
    y_tests = np.asarray(y_tests)
   
    runs_metrics = np.asarray(runs_metrics)
    runs_predictions = np.asarray(runs_predictions)
    mse_errors = np.asarray(mse_errors)
    correlations = np.asarray(correlations)
    spearman_correlations = np.asarray(spearman_correlations)
    voxels = np.asarray(voxel)
    voxels_predicteds = np.asarray(voxels_predicteds)

    mse_errors = np.asarray(mse_errors)
    correlations = np.asarray(correlations)


    stats_correlations = correlations[:, 0]
    pvalues_correlations = correlations[:, 1]
    
    stats_spearman_correlations = spearman_correlations[:, 0]
    pvalues_spearman_correlations = spearman_correlations[:, 1]
    

    
    metrics_df = pd.DataFrame(
        np.column_stack([stats_correlations, pvalues_correlations, mse_errors,voxels,stats_spearman_correlations,pvalues_spearman_correlations,runs_metrics]),
        columns=['pearson_correlation', 'pearson_pvalue', 'mse','voxel','spearman_correlation','spearman_pvalue','run']
    )
    
    predictions_df = pd.DataFrame(
        np.column_stack([predicteds, y_tests,voxels_predicteds,runs_predictions]),
        columns=['predicted', 'target', 'voxel','run']
    )

    return predictions_df, metrics_df

In [14]:
def compute_correlation(X,y,n_components=None):
    """
    Compute correlations for MolFormer across specified layers and voxels.

    Parameters:
    times (int): Number of cross-validation iterations.
    n_components (int): Number of components for dimensionality reduction.
    fmri_ev (DataFrame): fMRI event data.
    input_file_molformer (str): Base path for MolFormer input files.
    layers (list, optional): List of layers to process. Default is [13].

    Returns:
    tuple: Metrics and predictions for MolFormer.
    """
    metrics_molformer = []
    predictions_molformer = []

    # create dictionary keys as indices of fmri_all_voxels and values as fmri_all_voxels
    # data_dict = {'voxel_'+str(i):y[:,i] for i in range(y.shape[1])}
    # data_dict['CID'] = CIDs
    # fmri_df = pd.DataFrame(data_dict)
    
    
        
        
    
    # X: behavior/embeddings, y: fmri
    results = pipeline(X,
        y, times=15,n_components=n_components)

    df_predictions_molformer, df_metric_molformer = results



    df_metric_molformer["subject"]=subject
    df_predictions_molformer["subject"]=subject
    metrics_molformer.append(df_metric_molformer)
    predictions_molformer.append(df_predictions_molformer)
    
       
    return metrics_molformer, predictions_molformer

In [15]:
# for subject in [1,2,3]:
#     for roi in ['PirF','PirT','AMY','OFC']:
#         molformer_embeddings,pom_embeddings,behavior_ratings,fmri,_,_,_,_ = read_fmri_sagar(subject, roi)
# 
#         pca = PCA()
#         #
#         # Determine transformed features
#         #
#         X_train_pca = pca.fit_transform(molformer_embeddings)
#         #
#         # Determine explained variance using explained_variance_ration_ attribute
#         #
#         exp_var_pca = pca.explained_variance_ratio_
#         #
#         # Cumulative sum of eigenvalues; This will be used to create step plot
#         # for visualizing the variance explained by each principal component.
#         #
#         cum_sum_eigenvalues = np.cumsum(exp_var_pca)
#         #
#         # Create the visualization plot
#         #
#         plt.bar(range(0,len(exp_var_pca)), exp_var_pca, alpha=0.5, align='center', label='Individual explained variance')
#         plt.step(range(0,len(cum_sum_eigenvalues)), cum_sum_eigenvalues, where='mid',label='Cumulative explained variance')
#         plt.ylabel('Explained variance ratio')
#         plt.xlabel('Principal component index')
#         plt.legend(loc='best')
#         plt.tight_layout()
#         plt.show()

In [16]:
# layers_start =[1,1,1,1,1000,1000]
# layers_end = [9,7,13,13,1001,1001]
#  #rw<masks>nii = masks for PirF (rw.APC,nii), PirT (rw.PPC.nii), AMY (rwAmygdala.nii), OFC (rwOFC.nii)
#
# # for m,model in enumerate(['ChemBERT_ChEMBL_pretrained', 'ChemBERTa-zinc-base-v1','MoLFormer-XL-both-10pct'
# #                                      ,'SELFormer'
# #                                   ]):
# # for layer in range(layers_end[m]-1,layers_end[m]-3,-1):
# for subject in [1,2,3]:
#
#     for roi in ['PirF','PirT','AMY','OFC']:
#         print(f"Processing subject {subject}, roi {roi}")
#
#         fmri = read_fmri_singletrial(subject,roi)
#
#         behavior_ratings =  np.load(f'../results/ratings_{subject}.npy')
#
#         # print(f"Processingmodel {model}, layer {layer}")
#         # embeddings = np.load(f'../results/embeddings_{model}_{subject}_{layer}.npy')
#         metrics, predictions = compute_correlation(behavior_ratings,fmri,n_components=None)
#         if metrics is None:
#             continue
#         metrics = pd.concat(metrics)
#         predictions =  pd.concat(predictions)
#         predictions.to_csv(f"{base_dir}/fmri/predictions_avgsingletrial/subj_{subject}_{roi}_behavior_0_predictions.csv",index=False)
#         metrics.to_csv(f"{base_dir}/fmri/metrics_avgsingletrial/subj_{subject}_{roi}_behavior_0_metrics.csv",index=False)


In [21]:
layers_start =[1,1,1,1,1000,1000]
layers_end = [9,7,13,13,1001,1001]
 #rw<masks>nii = masks for PirF (rw.APC,nii), PirT (rw.PPC.nii), AMY (rwAmygdala.nii), OFC (rwOFC.nii)

for m,model in enumerate([
    # 'ChemBERT_ChEMBL_pretrained', 'ChemBERTa-zinc-base-v1','MoLFormer-XL-both-10pct',
                                     'SELFormer'
                                  ]):
    # for layer in range(layers_end[m]-1,layers_end[m]-2,-1):
    for layer in range(layers_end[3]-1,layers_end[3]-2,-1):
        for subject in [1,2,3]:

            for roi in ['PirF','PirT','AMY','OFC']:
                print(f"Processing subject {subject}, roi {roi}")

                fmri = read_fmri_singletrial(subject,roi)

                # behavior_ratings =  np.load(f'../results/ratings_{subject}.npy')

                print(f"Processingmodel {model}, layer {layer}")
                embeddings = np.load(f'../results/embeddings_{model}_{subject}_{layer}.npy')
                embeddings=np.random.rand(embeddings.shape[0],embeddings.shape[1])
                metrics, predictions = compute_correlation(embeddings,fmri,n_components=None)
                if metrics is None:
                    continue
                metrics = pd.concat(metrics)
                predictions =  pd.concat(predictions)
                predictions.to_csv(f"{base_dir}/fmri/predictions_avgsingletrial/subj_{subject}_{roi}_{model}_{layer}_predictions.csv",index=False)
                metrics.to_csv(f"{base_dir}/fmri/metrics_avgsingletrial/subj_{subject}_{roi}_{model}_{layer}_metrics.csv",index=False)


Processing subject 1, roi PirF
Processingmodel SELFormer, layer 12
Processing run 0
Processing run 1
Processing run 2
Processing run 3
Processing run 4
Processing run 5
Processing run 6
Processing run 7
Processing run 8
Processing run 9
Processing run 10
Processing run 11
Processing run 12
Processing run 13
Processing run 14
Processing subject 1, roi PirT
Processingmodel SELFormer, layer 12
Processing run 0
Processing run 1
Processing run 2
Processing run 3


KeyboardInterrupt: 

In [19]:
layers_start =[1000,1000]
layers_end = [1001,1001]
 #rw<masks>nii = masks for PirF (rw.APC,nii), PirT (rw.PPC.nii), AMY (rwAmygdala.nii), OFC (rwOFC.nii)

for m,model in enumerate([
    # 'ChemBERT_ChEMBL_pretrained', 'ChemBERTa-zinc-base-v1'
    # 'MoLFormer-XL-both-10pct',
    #                                  'SELFormer',
    'openpom','behavior'
                                  ]):
    # for layer in range(layers_end[m]-1,layers_end[m]-2,-1):
    for layer in range(layers_end[m]-1,layers_end[m]-2,-1):
        for subject in [1,2,3]:

            for roi in ['PirF','PirT','AMY','OFC']:
                print(f"Processing subject {subject}, roi {roi}")

                fmri = read_fmri_singletrial(subject,roi)

                # behavior_ratings =  np.load(f'../results/ratings_{subject}.npy')

                print(f"Processingmodel {model}, layer {layer}")
                embeddings = np.load(f'../results/embeddings_{model}_{subject}_{layer}.npy')
                embeddings=np.random.rand(embeddings.shape[0],embeddings.shape[1])
                metrics, predictions = compute_correlation(embeddings,fmri,n_components=None)
                if metrics is None:
                    continue
                metrics = pd.concat(metrics)
                predictions =  pd.concat(predictions)
                predictions.to_csv(f"{base_dir}/fmri/predictions_avgsingletrial/subj_{subject}_{roi}_{model}random_{layer}_predictions.csv",index=False)
                metrics.to_csv(f"{base_dir}/fmri/metrics_avgsingletrial/subj_{subject}_{roi}_{model}random_{layer}_metrics.csv",index=False)


Processing subject 1, roi PirF
Processingmodel openpom, layer 1000


FileNotFoundError: [Errno 2] No such file or directory: '../results/embeddings_openpom_1_1000.npy'

In [18]:
# for subject in range(2,4):
#     for roi in ['PirF','PirT','AMY','OFC']:
#         print(f"Processing subject {subject}, roi {roi}")
#         behavior_ratings =  np.load(f'../results/ratings_{subject}.npy')
#         fmri = read_fmri_singletrial(subject,roi)
#         metrics, predictions = compute_correlation(behavior_ratings,fmri,n_components=None)
#         metrics = pd.concat(metrics)
#         predictions =  pd.concat(predictions)
#         metrics.to_csv(f"{base_dir}/fmri/metrics_avgsingletrial/subj_{subject}_{roi}_{'behavior'}_{1000}_metrics.csv",index=False)
#         predictions.to_csv(f"{base_dir}/fmri/predictions_avgsingletrial/subj_{subject}_{roi}_{'behavior'}_{1000}_predictions.csv",index=False)
#
# for subject in range(1,4):
#     for roi in ['PirF','PirT','AMY','OFC']:
#         print(f"Processing subject {subject}, roi {roi}")
#         CIDs = pd.read_csv(f'../results/CIDs_smiles_{subject}.csv')['CIDs']
#         Pom_embeddings = read_pom(CIDs,subject)
#         fmri = read_fmri(subject,roi)
#         metrics, predictions = compute_correlation(Pom_embeddings,fmri,n_components=30)
#         metrics = pd.concat(metrics)
#         predictions =  pd.concat(predictions)
#         metrics.to_csv(f"{base_dir}/fmri/metrics_avgsingletrial/subj_{subject}_{roi}_{'openpom'}_{1000}_metrics.csv",index=False)
#         predictions.to_csv(f"{base_dir}/fmri/predictions_avgsingletrial/subj_{subject}_{roi}_{'openpom'}_{1000}_predictions.csv",index=False)

Processing subject 2, roi PirF
Processing run 0
Processing run 1
Processing run 2
Processing run 3
Processing run 4
Processing run 5
Processing run 6
Processing run 7
Processing run 8
Processing run 9
Processing run 10
Processing run 11
Processing run 12
Processing run 13
Processing run 14
Processing subject 2, roi PirT
Processing run 0


/var/folders/s8/bznqh0v13ddg31lz919tkrmc0000gn/T/ipykernel_76824/2694039457.py:23: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  correlations = [scipy.stats.pearsonr(predicted[:, i], y[:, i]) for i in range(y.shape[1])]
/var/folders/s8/bznqh0v13ddg31lz919tkrmc0000gn/T/ipykernel_76824/2694039457.py:24: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  spearman_correlations = [scipy.stats.spearmanr(predicted[:, i], y[:, i]) for i in range(y.shape[1])]
/var/folders/s8/bznqh0v13ddg31lz919tkrmc0000gn/T/ipykernel_76824/2694039457.py:23: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  correlations = [scipy.stats.pearsonr(predicted[:, i], y[:, i]) for i in range(y.shape[1])]
/var/folders/s8/bznqh0v13ddg31lz919tkrmc0000gn/T/ipykernel_76824/2694039457.py:24: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  s

Processing run 1


/var/folders/s8/bznqh0v13ddg31lz919tkrmc0000gn/T/ipykernel_76824/2694039457.py:24: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  spearman_correlations = [scipy.stats.spearmanr(predicted[:, i], y[:, i]) for i in range(y.shape[1])]
/var/folders/s8/bznqh0v13ddg31lz919tkrmc0000gn/T/ipykernel_76824/2694039457.py:23: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  correlations = [scipy.stats.pearsonr(predicted[:, i], y[:, i]) for i in range(y.shape[1])]
/var/folders/s8/bznqh0v13ddg31lz919tkrmc0000gn/T/ipykernel_76824/2694039457.py:24: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  spearman_correlations = [scipy.stats.spearmanr(predicted[:, i], y[:, i]) for i in range(y.shape[1])]
/var/folders/s8/bznqh0v13ddg31lz919tkrmc0000gn/T/ipykernel_76824/2694039457.py:23: ConstantInputWarning: An input array is constant; the correlation coefficient is not de

Processing run 2


...

Processing run 13


/var/folders/s8/bznqh0v13ddg31lz919tkrmc0000gn/T/ipykernel_76824/2694039457.py:23: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  correlations = [scipy.stats.pearsonr(predicted[:, i], y[:, i]) for i in range(y.shape[1])]
/var/folders/s8/bznqh0v13ddg31lz919tkrmc0000gn/T/ipykernel_76824/2694039457.py:24: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  spearman_correlations = [scipy.stats.spearmanr(predicted[:, i], y[:, i]) for i in range(y.shape[1])]
/var/folders/s8/bznqh0v13ddg31lz919tkrmc0000gn/T/ipykernel_76824/2694039457.py:23: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  correlations = [scipy.stats.pearsonr(predicted[:, i], y[:, i]) for i in range(y.shape[1])]
/var/folders/s8/bznqh0v13ddg31lz919tkrmc0000gn/T/ipykernel_76824/2694039457.py:24: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  s

Processing run 14


/var/folders/s8/bznqh0v13ddg31lz919tkrmc0000gn/T/ipykernel_76824/2694039457.py:23: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  correlations = [scipy.stats.pearsonr(predicted[:, i], y[:, i]) for i in range(y.shape[1])]
/var/folders/s8/bznqh0v13ddg31lz919tkrmc0000gn/T/ipykernel_76824/2694039457.py:24: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  spearman_correlations = [scipy.stats.spearmanr(predicted[:, i], y[:, i]) for i in range(y.shape[1])]
/var/folders/s8/bznqh0v13ddg31lz919tkrmc0000gn/T/ipykernel_76824/2694039457.py:23: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  correlations = [scipy.stats.pearsonr(predicted[:, i], y[:, i]) for i in range(y.shape[1])]
/var/folders/s8/bznqh0v13ddg31lz919tkrmc0000gn/T/ipykernel_76824/2694039457.py:24: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  s

Processing subject 1, roi PirF
Processing run 0
Processing run 1
Processing run 2
Processing run 3
Processing run 4
Processing run 5
Processing run 6
Processing run 7
Processing run 8
Processing run 9
Processing run 10
Processing run 11
Processing run 12
Processing run 13
Processing run 14
Processing subject 1, roi PirT
Processing run 0
Processing run 1
Processing run 2
Processing run 3
Processing run 4
Processing run 5
Processing run 6
Processing run 7
Processing run 8
Processing run 9
Processing run 10
Processing run 11
Processing run 12
Processing run 13
Processing run 14
Processing subject 1, roi AMY
Processing run 0
Processing run 1
Processing run 2
Processing run 3
Processing run 4
Processing run 5
Processing run 6
Processing run 7
Processing run 8
Processing run 9
Processing run 10
Processing run 11
Processing run 12
Processing run 13
Processing run 14
Processing subject 1, roi OFC
Processing run 0
Processing run 1
Processing run 2
Processing run 3
Processing run 4
Processing ru

In [30]:

# #read maximum and minimum values of fmri for each voxel, subject and roi
# for subject in range(1,4):
#     for roi in ['PirF','PirT','AMY','OFC']:
#
#         print(dffmri.shape)

Processing subject 1, roi PirF
(101, 3)
Processing subject 1, roi PirT
(142, 3)
Processing subject 1, roi AMY
(339, 3)
Processing subject 1, roi OFC
(1256, 3)
Processing subject 2, roi PirF
(93, 3)
Processing subject 2, roi PirT
(72, 3)
Processing subject 2, roi AMY
(340, 3)
Processing subject 2, roi OFC
(1494, 3)
Processing subject 3, roi PirF
(29, 3)
Processing subject 3, roi PirT
(135, 3)
Processing subject 3, roi AMY
(231, 3)
Processing subject 3, roi OFC
(726, 3)


In [31]:
dffmri

,max_fmri,min_fmri,voxel
0,0.081701,-0.090166,0
1,0.130978,-0.061062,1
2,0.092289,-0.024385,2
3,0.113440,-0.039112,3
4,0.125813,-0.086791,4
...,...,...,...
721,0.119607,-0.065747,721
722,0.135975,-0.059789,722
723,0.070664,-0.011915,723
724,0.116723,-0.007287,724
